# Langchain Demo
Test how `langchain` can be used to build no-code LLMOps. Proof of Concept.


## Setting up Enviornment

In [1]:
%pip install langchain
%pip install openai

# use this to set API Key
#!echo "Fill in here openAI API" > api_key_private.txt




[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 77.0/77.0 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 78.3/78.3 kB 4.3 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
'export' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [11]:
"""Reading API from private file"""

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

try:
    with open('api_key_private.txt', 'r') as file:
        # Read the entire content of the file into a string
        api_key = file.read()

        # Print the string read from the file
        # print("String read from the file:")
        # print(api_key)
except FileNotFoundError:
    print("The file 'api_key.txt' was not found.")
except Exception as e:
    print("An error occurred:", str(e))

## Simple QA

In [19]:
llm = OpenAI(openai_api_key=api_key)
chat_model = ChatOpenAI(openai_api_key=api_key)

llm.predict("hi! Tell me about the weather")

'\n\nIt is currently sunny and pleasant outside. The temperature is around 65 degrees Fahrenheit. There is a light breeze in the air.'

In [22]:
chat_model.predict("hi!")

'Hello! How can I assist you today?'

In [21]:
from langchain.schema import HumanMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]

llm.predict_messages(messages)

chat_model.predict_messages(messages)

AIMessage(content='Colorful Threads')

## User-define Variable
*template*

In [23]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

In [24]:
from langchain.prompts.chat import ChatPromptTemplate

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]

## Parser
We can create our own parser

In [26]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")

['hi', 'bye']

## Basic Workflow Using Chains 
chain input -> *PromptTemplate + LLM + OutputParser* -> list

In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])
chain = chat_prompt | chat_model | CommaSeparatedListOutputParser() #chaining
chain.invoke({"text": "colors"})

['red', 'blue', 'green', 'yellow', 'purple']

What the No-code LLM do is to modularize these **parser/LLMs/thirdparty_plugin** into components and build graphs out of it.
LangFlow is just defining their graph over Python. That parse json file into a graph and generate langchain code.
flowise focus on the UI and integration beside the graph.



**Mediapipe** (Google's CV package) also adopt this, allow we to create user-defined Graph.


Below is an example of a graph (mediapipe):

    # This graph named main_pass_throughcals_nosubgraph.pbtxt contains 4
    # passthrough calculators.
    input_stream: "in"
    output_stream: "out"
    node {
        calculator: "PassThroughCalculator"
        input_stream: "in"
        output_stream: "out1"
    }
    node {
        calculator: "PassThroughCalculator"
        input_stream: "out1"
        output_stream: "out2"
    }
    node {
        calculator: "PassThroughCalculator"
        input_stream: "out2"
        output_stream: "out3"
    }
    node {
        calculator: "PassThroughCalculator"
        input_stream: "out3"
        output_stream: "out"
    }
